# TD3-Portfoliomanagement für den Dow Jones 30

Dieses Notebook zeigt, wie sich mit dem FinRL-Framework und dem TD3-Algorithmus (Twin Delayed DDPG) eine dynamische Portfolio-Allokation für den Dow Jones 30 aufbauen lässt. Alle Abschnitte sind kommentiert, um auch Einsteiger*innen bei jedem Schritt mitzunehmen.


In [47]:
# Paketinstallation (ggf. beim ersten Start ausführen)
# Hinweis (Deutsch): Die Installation kann ein paar Minuten dauern.
# Achtung: FinRL wird direkt aus dem GitHub-Master installiert, da dort macOS-kompatible Abhängigkeiten gepflegt werden.
%pip install -q "git+https://github.com/AI4Finance-Foundation/FinRL.git@master" ta


Note: you may need to restart the kernel to use updated packages.


In [ ]:
# Basisimporte und Verzeichnisse vorbereiten (Kommentare auf Deutsch)
import warnings
warnings.filterwarnings("ignore")

from pathlib import Path
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os

from finrl import config
from finrl.config import INDICATORS


from finrl.meta.preprocessor.yahoodownloader import YahooDownloader
from finrl.meta.preprocessor.preprocessors import data_split
from finrl.meta.env_portfolio_allocation.env_portfolio import StockPortfolioEnv
from finrl.agents.stablebaselines3.models import DRLAgent
from finrl.plot import backtest_stats, backtest_plot, get_daily_return, get_baseline
from finrl.meta.preprocessor.preprocessors import FeatureEngineer
from finrl.plot import convert_daily_return_to_pyfolio_ts

from pypfopt.efficient_frontier import EfficientFrontier
from pypfopt import risk_models, expected_returns

from pyfolio import timeseries






In [59]:
if not os.path.exists("./" + config.DATA_SAVE_DIR):
    os.makedirs("./" + config.DATA_SAVE_DIR)
if not os.path.exists("./" + config.TRAINED_MODEL_DIR):
    os.makedirs("./" + config.TRAINED_MODEL_DIR)
if not os.path.exists("./" + config.TENSORBOARD_LOG_DIR):
    os.makedirs("./" + config.TENSORBOARD_LOG_DIR)
if not os.path.exists("./" + config.RESULTS_DIR):
    os.makedirs("./" + config.RESULTS_DIR)

## Modellpfade für bereits trainierte Agenten

Damit wir nicht bei jedem Lauf erneut trainieren müssen, protokollieren wir hier die Speicherorte der zuletzt gesicherten Checkpoints je Algorithmus.


In [76]:
DOW_30_TICKER = [
    'MMM', 'AXP','AMGN','AMZN', 'AAPL', 'BA', 'CAT', 'CVX', 'CSCO', 
    'KO', 'DIS', 'GS', 'HD', 'HON', 'IBM', 'JNJ', 
    'JPM', 'MCD', 'MRK', 'MSFT','NVDA', 'NKE', 'PG', 'CRM','SHW', 'TRV', 
    'UNH', 'VZ', 'V', 'WMT'
]

In [77]:
# Dow-Jones-Konfiguration und Trainings-/Testzeiträume (Kommentare auf Deutsch)
ticker_list = DOW_30_TICKER


train_start_date = "2008-01-01"
train_end_date = "2020-12-31"

validate_start_date = "2021-01-01"
validate_end_date = "2022-12-31"

trade_start_date = "2022-01-01"
trade_end_date = "2025-11-01"

initial_capital = 1_000_000
transaction_cost_pct = 0.001  # 10 Basispunkte pro Trade
hmax = 100  # maximale Stückzahl pro Order
reward_scaling = 1e-4  
num_stock_shares = 1000


## Schritt 1: Daten- und Marktparameter festlegen

Wir definieren, welche Dow-Jones-30-Titel, welchen Zeitraum und welche Finanzindikatoren unser Agent für das Training und das anschließende Trading nutzen soll.


In [78]:
df_raw = YahooDownloader(
    start_date=train_start_date,
    end_date=trade_end_date,
    ticker_list= DOW_30_TICKER,
).fetch_data()
df_raw.to_csv('dow_30_data.csv')

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%********

Shape of DataFrame:  (134587, 8)


In [79]:
df_raw

df_raw.isnull().values.any()

np.False_

## Schritt 2: Datenbeschaffung & Feature Engineering

Wir nutzen den integrierten `DataProcessor` von FinRL, der Yahoo Finance ansteuert, sämtliche Dow-Jones-Kurse lädt und anschließend technische Indikatoren hinzufügt.


In [80]:
#tech_indicators = ['rsi','macd']

fe = FeatureEngineer(
    use_technical_indicator=True,
    tech_indicator_list=INDICATORS,
    use_vix=False,        
    use_turbulence=False,  
    user_defined_feature=False
)
df = fe.preprocess_data(df_raw)

df=df.sort_values(['date','tic'],ignore_index=True)
df.index = df.date.factorize()[0]

cov_list = []
return_list = []


lookback=252
for i in range(lookback,len(df.index.unique())):
  data_lookback = df.loc[i-lookback:i,:]
  price_lookback=data_lookback.pivot_table(index = 'date',columns = 'tic', values = 'close')
  return_lookback = price_lookback.pct_change().dropna()
  return_list.append(return_lookback)

  covs = return_lookback.cov().values 
  cov_list.append(covs)

  
df_cov = pd.DataFrame({'date':df.date.unique()[lookback:],'cov_list':cov_list,'return_list':return_list})
df = df.merge(df_cov, on='date')
df = df.sort_values(['date','tic']).reset_index(drop=True)


df


Successfully added technical indicators


,date,close,high,low,open,volume,tic,day,macd,boll_ub,boll_lb,rsi_30,cci_30,dx_30,close_30_sma,close_60_sma,cov_list,return_list
0,2008-12-31,2.559735,2.631413,2.559435,2.578329,607541200,AAPL,2,-0.081830,3.064709,2.431330,42.254776,-80.847429,16.129997,2.723836,2.834898,"[[0.0013489684484362565, 0.0004284133291526246...",tic AAPL AMGN AMZN ...
1,2008-12-31,39.332115,39.652222,38.862174,38.896227,6287200,AMGN,2,0.147362,40.147627,38.404895,51.060626,51.895814,10.431925,38.597457,38.170424,"[[0.0013489684484362565, 0.0004284133291526246...",tic AAPL AMGN AMZN ...
2,2008-12-31,2.564000,2.584500,2.495500,2.537000,155844000,AMZN,2,0.048231,2.712887,2.319763,49.073148,58.457892,4.441536,2.344533,2.466917,"[[0.0013489684484362565, 0.0004284133291526246...",tic AAPL AMGN AMZN ...
3,2008-12-31,14.254675,14.408365,13.762870,13.808976,9625600,AXP,2,-0.914660,18.229860,12.377336,42.554853,-74.811285,25.740060,15.391969,17.222408,"[[0.0013489684484362565, 0.0004284133291526246...",tic AAPL AMGN AMZN ...
4,2008-12-31,32.005890,32.290921,31.128298,31.195806,5443100,BA,2,-0.279800,32.174385,28.867830,47.440244,157.922054,5.392003,30.327214,32.389914,"[[0.0013489684484362565, 0.0004284133291526246...",tic AAPL AMGN AMZN ...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
122839,2025-10-31,344.135773,346.390513,342.629284,344.375203,1558200,SHW,4,-0.114875,349.007182,323.242812,50.008121,68.312113,40.070235,338.373908,349.850782,"[[0.0004226833371400604, 9.58381696081319e-05,...",tic AAPL AMGN AMZN ...
122840,2025-10-31,268.619995,271.100006,267.750000,268.320007,1900900,TRV,4,-1.574636,283.656000,258.556009,48.494234,-54.270578,2.088467,273.356336,272.521841,"[[0.0004226833371400604, 9.58381696081319e-05,...",tic AAPL AMGN AMZN ...
122841,2025-10-31,341.559998,346.279999,337.119995,344.390015,8390800,UNH,4,2.566168,374.210235,345.512769,50.206397,-128.399797,6.756073,356.012333,332.561928,"[[0.0004226833371400604, 9.58381696081319e-05,...",tic AAPL AMGN AMZN ...
122842,2025-10-31,39.740002,39.959999,38.779999,38.980000,52181800,VZ,4,-0.708339,41.363981,38.558197,43.848153,-77.341672,12.228777,40.887493,42.056723,"[[0.0004226833371400604, 9.58381696081319e-05,...",tic AAPL AMGN AMZN ...


In [81]:
train = data_split(df, start=train_start_date, end=train_end_date)
validate = data_split(df, start=validate_start_date, end=validate_end_date)
trade = data_split(df, start=trade_start_date, end=trade_end_date)

## Schritt 3: Handelsumgebung definieren

Wir erstellen zwei `StockTradingEnv`-Instanzen – eine für das Training und eine für das spätere Trading/Backtesting. Die Umgebung erhält Angaben zu Kapital, Transaktionskosten, Risiko-Kontrollen und zu den technischen Indikatoren.


In [82]:

stock_dim = len(train.tic.unique())

print(f"State-Dimension: {stock_dim} | Aktienanzahl: {stock_dim}")

env_kwargs = {
    "hmax": hmax,
    "initial_amount": initial_capital,
    "transaction_cost_pct": transaction_cost_pct,
    "state_space": stock_dim,
    "stock_dim": stock_dim,
    "tech_indicator_list": INDICATORS,
    "action_space": stock_dim,
    "reward_scaling": reward_scaling,
}

e_train_gym = StockPortfolioEnv(df=train, **env_kwargs)
e_validate_gym = StockPortfolioEnv(df=validate, **env_kwargs)
e_trade_gym = StockPortfolioEnv(df=trade, **env_kwargs)

State-Dimension: 29 | Aktienanzahl: 29


## Agenten konfigurieren


In [ ]:
agent = DRLAgent(env=e_train_gym)

model_path = {}

models =["ppo","ddpg","td3"]

for model_name in models:
    
    model = agent.get_model(model_name)
    trained_model = agent.train_model(model=model, 
                                    tb_log_name=model_name, 
                                    total_timesteps=50000
                                    )
    trained_model.save(f"trained_untuned_models/{model_name}_portfolio")
    
    model_path[model_name] = Path(f"trained_untuned_models/{model_name}_portfolio")



{'n_steps': 2048, 'ent_coef': 0.01, 'learning_rate': 0.00025, 'batch_size': 64}
Using cpu device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
----------------------------------
| time/              |           |
|    fps             | 1118      |
|    iterations      | 1         |
|    time_elapsed    | 1         |
|    total_timesteps | 2048      |
| train/             |           |
|    reward          | 5061658.0 |
|    reward_max      | 5061658.0 |
|    reward_mean     | 2633058.2 |
|    reward_min      | 810325.44 |
----------------------------------
begin_total_asset:1000000
end_total_asset:10697291.389483625
Sharpe:  1.16002814164097
----------------------------------------
| rollout/                |            |
|    ep_len_mean          | 3.02e+03   |
|    ep_rew_mean          | 1.24e+10   |
| time/                   |            |
|    fps                  | 931        |
|    iterations           | 2          |
|    time_elapsed         | 4   

KeyboardInterrupt: 

In [ ]:

for model in models:
    
    MODEL_PATHS = {
        {model}: Path(f"trained_models/{model}_portfolio.zip"),
    }

    

# Baselines 


### DJI Buy & Hold

In [ ]:
baseline_df = get_baseline(
        ticker="^DJI", 
        start = trade_start_date,
        end =  trade_end_date)

baseline_df_stats = backtest_stats(baseline_df, value_col_name = 'close')
baseline_returns = get_daily_return(baseline_df, value_col_name="close")


dji_cumpod =(baseline_returns+1).cumprod()-1

[*********************100%***********************]  1 of 1 completed

Shape of DataFrame:  (962, 8)
Annual return          0.071158
Cumulative returns     0.300063
Annual volatility      0.153786
Sharpe ratio           0.524238
Calmar ratio           0.324317
Stability              0.820039
Max drawdown          -0.219408
Omega ratio            1.097346
Sortino ratio          0.757259
Skew                        NaN
Kurtosis                    NaN
Tail ratio             0.959405
Daily value at risk   -0.019055
dtype: float64


### Markowitz-Portfolio mit rollierendem Mean-Variance-Ansatz



In [ ]:

# Markowitz Mean-Variance mit rollierendem Fenster und quartalsweisem Rebalancing (Deutsch kommentiert)
MARKOWITZ_LOOKBACK = 252  # 1 Jahr historische Daten für Schätzung
MARKOWITZ_TXN_COST = 0.001  # 0,1% Transaktionskosten
MARKOWITZ_REBAL_FREQ = "QS"  # Quartalsanfang (Quarter Start)

# Preisdaten für gesamten Zeitraum (inkl. Historie) vorbereiten
prices_all = (
    df.pivot_table(index="date", columns="tic", values="close")
      .sort_index()
      .ffill()  # Forward-Fill für fehlende Werte (trägt letzten bekannten Preis vor)
)
prices_all.index = pd.to_datetime(prices_all.index)
prices_trade = prices_all.loc[trade_start_date:trade_end_date]

# KORREKTUR 1: Nur Spalten entfernen, die ALLE Werte fehlen haben (nicht "any")
# Einzelne NaN sollten durch ffill bereits behoben sein
prices_trade = prices_trade.dropna(axis=1, how="all")


def run_markowitz_rolling_portfolio(
    prices_trade_period: pd.DataFrame,
    prices_full_history: pd.DataFrame,
    initial_capital: float,
    lookback_days: int = MARKOWITZ_LOOKBACK,
    transaction_cost: float = MARKOWITZ_TXN_COST,
    rebalance_frequency: str = MARKOWITZ_REBAL_FREQ,
):
    """
    Simuliert ein Markowitz-Portfolio mit rollierendem Schätzfenster.
    
    Args:
        prices_trade_period: Preise für den Trade-Zeitraum
        prices_full_history: Alle verfügbaren historischen Preise (für Lookback)
        initial_capital: Startkapital
        lookback_days: Anzahl Handelstage für rollierende Schätzung
        transaction_cost: Transaktionskosten als Dezimalzahl (0.001 = 0,1%)
        rebalance_frequency: Pandas-Frequenz für Rebalancing (z.B. "QS" für Quartalsanfang)
    
    Returns:
        df_portfolio_value: DataFrame mit täglichen Portfoliowerten und Returns
        df_rebalancing: DataFrame mit Rebalancing-Ereignissen und Gewichten
    """
    returns_trade = prices_trade_period.pct_change().dropna(how="all")
    if returns_trade.empty:
        raise ValueError("Keine Handelsdaten für den angegebenen Zeitraum.")
    
    # Rebalancing-Termine bestimmen (Quartalsanfänge)
    rebal_dates = (
        returns_trade.index.to_series()
        .resample(rebalance_frequency)
        .first()
        .dropna()
        .tolist()
    )
    
    # Ersten Handelstag hinzufügen, falls nicht bereits enthalten
    first_trade_day = returns_trade.index[0]
    if not rebal_dates or first_trade_day < rebal_dates[0]:
        rebal_dates.insert(0, first_trade_day)
    
    rebal_dates = [d for d in rebal_dates if d <= returns_trade.index[-1]]
    rebal_dates = sorted(set(rebal_dates))
    
    portfolio_value = initial_capital
    prev_weights = pd.Series(0.0, index=prices_trade_period.columns)
    
    value_records = []
    rebalancing_records = []
    
    for idx, rebal_date in enumerate(rebal_dates):
        # KORREKTUR 4: Look-Ahead Bias vermeiden!
        # Historisches Fenster MUSS STRIKT VOR dem Rebalancing-Tag enden
        # Am 03.01. kennen wir nur Daten bis 02.01. (Schlusskurs von gestern)
        hist_end_date = rebal_date - pd.Timedelta(days=1)
        hist_window = prices_full_history.loc[:hist_end_date, prices_trade_period.columns].tail(lookback_days)
        
       
        
        # Mean-Variance-Optimierung mit pypfopt (nur auf Daten BIS GESTERN)
        try:
            mu = expected_returns.mean_historical_return(hist_window, frequency=252)
            S = risk_models.sample_cov(hist_window, frequency=252)
            ef = EfficientFrontier(mu, S)
            ef.max_sharpe()  # Portfolio mit maximaler Sharpe Ratio
            weights_dict = ef.clean_weights()
            weights = pd.Series(weights_dict)
        except Exception as e:
            print(f"[Fehler] Optimierung fehlgeschlagen für {rebal_date.date()}: {e}")
            continue
        
        weights = weights.reindex(prices_trade_period.columns).fillna(0.0)
        
        
        # Transaktionskosten berechnen (Turnover = Summe der absoluten Gewichtsänderungen)
        turnover = (weights - prev_weights).abs().sum()
        txn_cost_value = turnover * transaction_cost * portfolio_value
        portfolio_value -= txn_cost_value
        
        prev_weights = weights.copy()
        
        rebalancing_records.append({
            "date": rebal_date,
            "transaction_cost": txn_cost_value,
            "turnover": turnover,
            "portfolio_value_after_costs": portfolio_value,
            **weights.to_dict(),
        })
        
        # Return-Periode: JETZT können wir ab rebal_date handeln (ex-ante korrekt!)
        if idx < len(rebal_dates) - 1:
            period_mask = (returns_trade.index >= rebal_date) & (returns_trade.index < rebal_dates[idx + 1])
        else:
            period_mask = returns_trade.index >= rebal_date
        
        period_rets = returns_trade.loc[period_mask]
        
        if period_rets.empty:
            continue
     
        portfolio_daily_returns = (period_rets * weights).sum(axis=1)
        
        # Berechne kumulative Portfolio-Werte durch kumulative Multiplikation
        cumulative_factors = (1 + portfolio_daily_returns).cumprod()
        portfolio_values = portfolio_value * cumulative_factors
        
        # Portfolio-Wert für nächste Iteration aktualisieren
        portfolio_value = portfolio_values.iloc[-1]
        
        # Records erstellen
        for dt, daily_ret, pv in zip(period_rets.index, portfolio_daily_returns, portfolio_values):
            value_records.append({
                "date": dt,
                "daily_return": daily_ret,
                "portfolio_value": pv,
            })
    
    df_portfolio = pd.DataFrame(value_records)
    df_rebal = pd.DataFrame(rebalancing_records)
    
    if not df_portfolio.empty:
        df_portfolio["date"] = pd.to_datetime(df_portfolio["date"])
        df_portfolio.sort_values("date", inplace=True)
    
    if not df_rebal.empty:
        df_rebal["date"] = pd.to_datetime(df_rebal["date"])
        df_rebal.sort_values("date", inplace=True)
    
    return df_portfolio, df_rebal


# Markowitz-Strategie ausführen
df_markowitz_portfolio, df_markowitz_rebalancing = run_markowitz_rolling_portfolio(
    prices_trade_period=prices_trade,
    prices_full_history=prices_all,
    initial_capital=initial_capital,
)

# Ergebnisse ausgeben
if df_markowitz_portfolio.empty:
    print("❌ Markowitz-Strategie lieferte keine Ergebnisse.")
else:
    final_value_mw = df_markowitz_portfolio["portfolio_value"].iloc[-1]
    cum_return_mw = (final_value_mw / initial_capital) - 1
    
    markowitz_returns = df_markowitz_portfolio.set_index("date")["daily_return"].dropna()
    sharpe_mw = np.nan
    if not markowitz_returns.empty and markowitz_returns.std() != 0:
        sharpe_mw = (markowitz_returns.mean() / markowitz_returns.std()) * np.sqrt(252)
    
    print(f"📊 Markowitz Mean-Variance Portfolio (Max Sharpe) - KORRIGIERT")
    print(f"   Zeitraum: {trade_start_date} bis {trade_end_date}")
    print(f"   Finaler Portfoliowert: ${final_value_mw:,.2f}")
    print(f"   Kumulative Rendite: {cum_return_mw:.2%}")
    if not np.isnan(sharpe_mw):
        print(f"   Sharpe Ratio (annualisiert): {sharpe_mw:.3f}")
    
    print(f"\n📈 Letzte 5 Portfoliowerte:")
    print(df_markowitz_portfolio[["date", "daily_return", "portfolio_value"]].tail())
    
    print(f"\n📊 Anzahl Trading-Tage: {len(df_markowitz_portfolio)}")
    
    if not df_markowitz_rebalancing.empty:
        num_rebal = len(df_markowitz_rebalancing)
        total_txn_costs = df_markowitz_rebalancing["transaction_cost"].sum()
        print(f"\n🔄 Rebalancing-Statistiken:")
        print(f"   Anzahl Rebalancings: {num_rebal}")
        print(f"   Gesamte Transaktionskosten: ${total_txn_costs:,.2f}")
        print(f"\n   Letzte 2 Rebalancing-Events (Gewichte):")
        display_cols = ["date", "transaction_cost", "turnover", "portfolio_value_after_costs"]
        display_cols_available = [c for c in display_cols if c in df_markowitz_rebalancing.columns]
        print(df_markowitz_rebalancing[display_cols_available].tail(2))


Finales Portfoliovermögen: 1,670,736.76 USD
Kumulative Rendite seit 2022-01-01: 67.07%
Approx. Sharpe Ratio (täglich * sqrt(252)): 0.80

Letzte 5 Portfoliowerte:
          date  daily_return  portfolio_value
955 2025-10-27      0.006461     1.670917e+06
956 2025-10-28     -0.002319     1.667043e+06
957 2025-10-29     -0.002863     1.662271e+06
958 2025-10-30      0.005626     1.671622e+06
959 2025-10-31     -0.000530     1.670737e+06

Letzte 2 Rebalancings und Gewichte:
         date  transaction_cost  portfolio_value_post_costs  AAPL  AMGN  AMZN  \
14 2025-07-01       1876.264173                1.618414e+06   0.0   0.0   0.0   
15 2025-10-01       2723.437720                1.641487e+06   0.0   0.0   0.0   

    AXP       BA  CAT  CRM  ...  MRK  MSFT  NKE     NVDA   PG  SHW     TRV  \
14  0.0  0.00000  0.0  0.0  ...  0.0   0.0  0.0  0.00000  0.0  0.0  0.0232   
15  0.0  0.03593  0.0  0.0  ...  0.0   0.0  0.0  0.07927  0.0  0.0  0.0000   

    UNH   VZ      WMT  
14  0.0  0.0  0.26642 

### Equal Weight DJI


In [ ]:
# Equal-Weight Portfolio (Buy-and-Hold, kein Rebalancing)
returns_trade = prices_trade.pct_change().dropna(how="all")

# Alle Aktien gleich gewichten (1/N)
n_stocks = len(prices_trade.columns)
equal_weights = pd.Series(1.0 / n_stocks, index=prices_trade.columns)

# Portfolio Returns berechnen (gewichteter Durchschnitt der Returns)
equal_weight_returns = (returns_trade * equal_weights).sum(axis=1)

# Portfolio-Wert über Zeit
initial_value = initial_capital
equal_weight_values = initial_value * (1 + equal_weight_returns).cumprod()

# DataFrame erstellen
df_equal_weight = pd.DataFrame({
    "date": equal_weight_returns.index,
    "daily_return": equal_weight_returns.values,
    "portfolio_value": equal_weight_values.values
})

# Ergebnisse
final_value_eq = df_equal_weight["portfolio_value"].iloc[-1]
cum_return_eq = (final_value_eq / initial_capital) - 1
sharpe_eq = (equal_weight_returns.mean() / equal_weight_returns.std()) * np.sqrt(252)

print(f"📊 Equal-Weight Portfolio (Buy-and-Hold)")
print(f"   Zeitraum: {trade_start_date} bis {trade_end_date}")
print(f"   Finaler Portfoliowert: ${final_value_eq:,.2f}")
print(f"   Kumulative Rendite: {cum_return_eq:.2%}")
print(f"   Sharpe Ratio (annualisiert): {sharpe_eq:.3f}")
print(f"   Anzahl Trading-Tage: {len(df_equal_weight)}")


## Rolling-Window-Evaluierung mit Walk-Forward-Training (Quartalsweise)

**Wichtig**: Dies ist ein echtes Walk-Forward-Setup! In jedem Quartal wird das Modell NEU TRAINIERT auf allen bis dahin verfügbaren Daten (Expanding Window), und dann auf dem nächsten Quartal evaluiert. Dies ist fair im Vergleich zu Markowitz, das sich auch quartalsweise neu anpasst.

**Methodik**: 
- Q1: Trainiere auf [2008-2021] → Teste auf Q1 2022
- Q2: Trainiere auf [2008-Q1 2022] → Teste auf Q2 2022
- Q3: Trainiere auf [2008-Q2 2022] → Teste auf Q3 2022
- usw.

Dies simuliert realistisch, wie ein Fondsmanager seine Modelle regelmäßig aktualisieren würde.


In [ ]:
# Walk-Forward Rolling-Window-Evaluierung der DRL-Agenten (Quartalsweise)
# Alle Kommentare auf Deutsch für besseres Verständnis
# WICHTIG: Modelle werden in jedem Quartal NEU TRAINIERT (Expanding Window)

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime, timedelta
from stable_baselines3 import PPO, DDPG, TD3
import os

# Erstelle Ordner für die trainierten Rolling-Window-Modelle
if not os.path.exists("./rolling_window_models"):
    os.makedirs("./rolling_window_models")

def create_quarterly_windows(start_date, end_date):
    """
    Erstellt quartalsweise Zeitfenster für die Rolling-Window-Evaluierung.
    
    Args:
        start_date: Startdatum des Evaluierungszeitraums (String im Format "YYYY-MM-DD")
        end_date: Enddatum des Evaluierungszeitraums (String im Format "YYYY-MM-DD")
    
    Returns:
        Liste von Tupeln (start, end) für jedes Quartal
    """
    # Konvertiere zu pandas Timestamps
    start = pd.Timestamp(start_date)
    end = pd.Timestamp(end_date)
    
    # Erstelle quartalsweise Zeitstempel
    quarters = pd.date_range(start=start, end=end, freq='QS')
    
    # Wenn der Starttermin nicht am Quartalsanfang liegt, füge ihn hinzu
    if start not in quarters:
        quarters = quarters.insert(0, start)
    
    # Erstelle Fenster (Paare von Start- und Enddatum)
    windows = []
    for i in range(len(quarters) - 1):
        windows.append((quarters[i], quarters[i+1]))
    
    # Füge das letzte Fenster bis zum Enddatum hinzu
    if quarters[-1] < end:
        windows.append((quarters[-1], end))
    
    return windows


def train_and_evaluate_walk_forward(model_type, train_data, test_data, env_kwargs, 
                                     window_idx, window_name, timesteps=30000):
    """
    WALK-FORWARD: Trainiert ein Modell auf train_data und evaluiert es auf test_data.
    
    Args:
        model_type: Typ des Modells ("ppo", "ddpg", "td3")
        train_data: Daten für das Training (Expanding Window - alle Daten bis jetzt)
        test_data: Daten für das Testing (aktuelles Quartal)
        env_kwargs: Umgebungsparameter
        window_idx: Index des Fensters (für Logging und Speichern)
        window_name: Name des Fensters (für Logging)
        timesteps: Anzahl der Trainings-Timesteps (Standard: 30000 für schnelleres Training)
    
    Returns:
        Dictionary mit Performance-Metriken
    """
    # SCHRITT 1: TRAINIERE das Modell auf allen verfügbaren historischen Daten
    print(f"      [1/2] Trainiere {model_type.upper()} auf {len(train_data['date'].unique())} Handelstagen...")
    
    # Erstelle Trainingsumgebung
    train_env = StockPortfolioEnv(df=train_data, **env_kwargs)
    
    # Initialisiere den Agenten
    agent = DRLAgent(env=train_env)
    
    # Hole das Modell und trainiere es
    model = agent.get_model(model_type)
    trained_model = agent.train_model(
        model=model, 
        tb_log_name=f"{model_type}_rolling_q{window_idx}", 
        total_timesteps=timesteps
    )
    
    # Speichere das Modell (für spätere Analysen)
    model_save_path = f"rolling_window_models/{model_type}_q{window_idx}"
    trained_model.save(model_save_path)
    
    # SCHRITT 2: EVALUIERE das frisch trainierte Modell auf dem Test-Quartal
    print(f"      [2/2] Evaluiere auf {len(test_data['date'].unique())} Test-Handelstagen...")
    
    # Erstelle Testumgebung
    test_env = StockPortfolioEnv(df=test_data, **env_kwargs)
    
    # Führe die Evaluierung durch
    obs = test_env.reset()
    done = False
    
    # Speichere die Zeitreihe der Portfoliowerte
    portfolio_values = [test_env.asset_memory[0]]
    daily_returns = []
    
    while not done:
        # Vorhersage der Aktion durch den FRISCH TRAINIERTEN Agenten
        action, _states = trained_model.predict(obs, deterministic=True)
        
        # Führe die Aktion in der Umgebung aus
        obs, reward, done, info = test_env.step(action)
        
        # Speichere den Portfoliowert
        if len(test_env.asset_memory) > len(portfolio_values):
            portfolio_values.append(test_env.asset_memory[-1])
            
            # Berechne tägliche Rendite
            if len(portfolio_values) > 1:
                daily_return = (portfolio_values[-1] - portfolio_values[-2]) / portfolio_values[-2]
                daily_returns.append(daily_return)
    
    # SCHRITT 3: Berechne Performance-Metriken
    initial_value = portfolio_values[0]
    final_value = portfolio_values[-1]
    total_return = (final_value - initial_value) / initial_value
    
    # Sharpe Ratio (annualisiert)
    if len(daily_returns) > 1 and np.std(daily_returns) > 0:
        daily_returns_array = np.array(daily_returns)
        sharpe_ratio = (np.mean(daily_returns_array) / np.std(daily_returns_array)) * np.sqrt(252)
    else:
        sharpe_ratio = 0.0
    
    # Maximum Drawdown
    cumulative_returns = np.array(portfolio_values) / initial_value
    running_max = np.maximum.accumulate(cumulative_returns)
    drawdown = (cumulative_returns - running_max) / running_max
    max_drawdown = np.min(drawdown)
    
    # Volatilität (annualisiert)
    volatility = np.std(daily_returns) * np.sqrt(252) if len(daily_returns) > 1 else 0.0
    
    results = {
        'window': window_name,
        'model': model_type,
        'initial_value': initial_value,
        'final_value': final_value,
        'total_return': total_return,
        'sharpe_ratio': sharpe_ratio,
        'max_drawdown': max_drawdown,
        'volatility': volatility,
        'num_days': len(portfolio_values) - 1,
        'portfolio_values': portfolio_values,
        'daily_returns': daily_returns,
        'dates': test_env.date_memory,
        'train_days': len(train_data['date'].unique()),
        'test_days': len(test_data['date'].unique())
    }
    
    return results


def run_walk_forward_evaluation(models_list, df_data, env_kwargs, 
                                 historical_start, test_start, test_end,
                                 training_timesteps=30000):
    """
    Führt die WALK-FORWARD-Evaluierung für alle Modelle durch.
    
    WICHTIG: In jedem Quartal wird das Modell NEU TRAINIERT auf allen Daten bis dahin
             (Expanding Window), und dann auf dem nächsten Quartal evaluiert.
    
    Args:
        models_list: Liste mit Modellnamen ["ppo", "ddpg", "td3"]
        df_data: Vollständiger Datensatz mit allen Features (von 2008 bis heute)
        env_kwargs: Umgebungsparameter
        historical_start: Startdatum der historischen Daten (z.B. "2008-01-01")
        test_start: Startdatum der Test-Periode (z.B. "2022-01-01")
        test_end: Enddatum der Test-Periode (z.B. "2025-11-01")
        training_timesteps: Anzahl Timesteps pro Training (Standard: 30000 für Geschwindigkeit)
    
    Returns:
        DataFrame mit allen Evaluierungsergebnissen, Dictionary mit Details
    """
    # Erstelle quartalsweise Test-Fenster
    test_windows = create_quarterly_windows(test_start, test_end)
    
    print("="*80)
    print("📊 WALK-FORWARD ROLLING-WINDOW-EVALUIERUNG")
    print("="*80)
    print(f"   Historischer Start: {historical_start}")
    print(f"   Test-Zeitraum: {test_start} bis {test_end}")
    print(f"   Anzahl Test-Quartale: {len(test_windows)}")
    print(f"   Modelle: {models_list}")
    print(f"   Training Timesteps pro Quartal: {training_timesteps}")
    print(f"\n   ⚠️  WICHTIG: Modelle werden in JEDEM Quartal NEU TRAINIERT!")
    print("="*80)
    
    # Speichere alle Ergebnisse
    all_results = []
    detailed_results = {}
    
    # Iteriere über alle Test-Quartale
    for i, (window_start, window_end) in enumerate(test_windows):
        window_name = f"Q{i+1}: {window_start.strftime('%Y-%m-%d')} bis {window_end.strftime('%Y-%m-%d')}"
        
        print(f"\n{'='*80}")
        print(f"🔍 QUARTAL {i+1}/{len(test_windows)}: {window_name}")
        print(f"{'='*80}")
        
        # EXPANDING WINDOW: Trainiere auf ALLEN Daten von Anfang bis EINEN TAG VOR window_start
        # FIX für Look-Ahead Bias: Training endet BEVOR das Test-Fenster beginnt
        train_end_date = (window_start - timedelta(days=1)).strftime('%Y-%m-%d')
        
        print(f"   📚 Training-Fenster: {historical_start} bis {train_end_date}")
        print(f"   🎯 Test-Fenster: {window_start.strftime('%Y-%m-%d')} bis {window_end.strftime('%Y-%m-%d')}")
        
        # WICHTIG: data_split FIX - füge +1 Tag zum End-Datum hinzu, um sicherzustellen,
        # dass der letzte Tag inkludiert wird (FinRL schneidet manchmal zu früh ab)
        test_end_safe = (window_end + timedelta(days=1)).strftime('%Y-%m-%d')
        
        # Hole Trainings- und Test-Daten
        try:
            train_data = data_split(df_data, start=historical_start, end=train_end_date)
            test_data = data_split(df_data, 
                                  start=window_start.strftime('%Y-%m-%d'),
                                  end=test_end_safe)
        except Exception as e:
            print(f"   ❌ Fehler beim Laden der Daten: {e}")
            continue
        
        # Überprüfe ob genügend Daten vorhanden sind
        if len(train_data) < 100:
            print(f"   ⚠️  Überspringe Fenster (zu wenig Trainingsdaten: {len(train_data)} Zeilen)")
            continue
        
        if len(test_data) < 10:
            print(f"   ⚠️  Überspringe Fenster (zu wenig Testdaten: {len(test_data)} Zeilen)")
            continue
        
        train_days = len(train_data['date'].unique())
        test_days = len(test_data['date'].unique())
        
        print(f"   ✅ Training: {train_days} Handelstage | Test: {test_days} Handelstage")
        
        # WALK-FORWARD: Trainiere und evaluiere jedes Modell
        for model_name in models_list:
            try:
                print(f"\n   🤖 Modell: {model_name.upper()}")
                
                results = train_and_evaluate_walk_forward(
                    model_type=model_name,
                    train_data=train_data,
                    test_data=test_data,
                    env_kwargs=env_kwargs,
                    window_idx=i+1,
                    window_name=window_name,
                    timesteps=training_timesteps
                )
                
                all_results.append(results)
                
                # Speichere detaillierte Ergebnisse
                key = f"{model_name}_q{i+1}"
                detailed_results[key] = results
                
                print(f"      ✅ ERGEBNIS: Rendite {results['total_return']:.2%} | " +
                      f"Sharpe {results['sharpe_ratio']:.3f} | " +
                      f"Max DD {results['max_drawdown']:.2%}")
                
            except Exception as e:
                print(f"      ❌ Fehler bei {model_name}: {str(e)}")
                import traceback
                traceback.print_exc()
                continue
    
    # Konvertiere Ergebnisse zu DataFrame
    if len(all_results) == 0:
        print("\n❌ Keine Ergebnisse! Überprüfe die Daten und Parameter.")
        return pd.DataFrame(), {}
    
    df_results = pd.DataFrame([{
        'Quartal': r['window'],
        'Modell': r['model'].upper(),
        'Start-Wert': r['initial_value'],
        'End-Wert': r['final_value'],
        'Rendite (%)': r['total_return'] * 100,
        'Sharpe Ratio': r['sharpe_ratio'],
        'Max Drawdown (%)': r['max_drawdown'] * 100,
        'Volatilität (%)': r['volatility'] * 100,
        'Handelstage': r['num_days'],
        'Train-Tage': r['train_days'],
        'Test-Tage': r['test_days']
    } for r in all_results])
    
    return df_results, detailed_results


# ============================================================================
# HAUPTAUSFÜHRUNG: WALK-FORWARD ROLLING-WINDOW-EVALUIERUNG
# ============================================================================
# 
# WICHTIG: Dies ist ein ECHTES Walk-Forward-Setup!
# - In jedem Quartal werden die Modelle NEU TRAINIERT (Expanding Window)
# - Training endet immer BEVOR das Test-Quartal beginnt (kein Look-Ahead Bias)
# - Dies ist ein fairer Vergleich mit Markowitz, das sich auch quartalsweise anpasst
#
# HINWEIS: Dies kann LANGE dauern (mehrere Stunden), da in jedem Quartal 
#          alle 3 Modelle neu trainiert werden müssen!
# ============================================================================

# Definiere welche Modelle evaluiert werden sollen
models_to_evaluate = ["ppo", "ddpg", "td3"]

# WICHTIG: Passe die Timesteps an (höher = bessere Modelle, aber langsamer)
# - 30000: Schnell (ca. 2-3 Min pro Modell pro Quartal) - für Tests
# - 50000: Mittel (wie im Original-Training) - empfohlen
# - 100000: Lang (bessere Qualität) - für finale Evaluierung
TRAINING_TIMESTEPS = 30000  # <-- Hier anpassen!

print("\n" + "="*80)
print("⚙️  KONFIGURATION")
print("="*80)
print(f"   Training Timesteps pro Quartal: {TRAINING_TIMESTEPS}")
print(f"   Geschätzte Dauer pro Quartal: {len(models_to_evaluate) * 3} Minuten")
print(f"   Modelle: {', '.join([m.upper() for m in models_to_evaluate])}")
print("="*80)

# Führe die Walk-Forward-Evaluierung durch
df_rolling_results, detailed_rolling_results = run_walk_forward_evaluation(
    models_list=models_to_evaluate,
    df_data=df,  # Vollständiger Datensatz mit allen Features (2008-2025)
    env_kwargs=env_kwargs,
    historical_start=train_start_date,  # 2008-01-01 (Beginn der historischen Daten)
    test_start=trade_start_date,  # 2022-01-01 (Start der Test-Quartale)
    test_end=trade_end_date,  # 2025-11-01 (Ende der Test-Quartale)
    training_timesteps=TRAINING_TIMESTEPS
)

# Zeige Ergebnisse
if not df_rolling_results.empty:
    print("\n" + "="*80)
    print("📊 WALK-FORWARD ERGEBNISSE - ZUSAMMENFASSUNG")
    print("="*80)
    print(df_rolling_results.to_string(index=False))
    
    # Speichere Ergebnisse als CSV
    df_rolling_results.to_csv("results/walk_forward_results.csv", index=False)
    print("\n💾 Ergebnisse gespeichert unter: results/walk_forward_results.csv")
    
    # Zeige Durchschnittswerte
    print("\n" + "="*80)
    print("📈 DURCHSCHNITTLICHE PERFORMANCE")
    print("="*80)
    avg_perf = df_rolling_results.groupby('Modell')[['Rendite (%)', 'Sharpe Ratio', 
                                                       'Max Drawdown (%)', 'Volatilität (%)']].mean()
    print(avg_perf.round(3))
else:
    print("\n❌ Keine Ergebnisse generiert. Überprüfe die Ausgabe oben für Fehler.")



⚙️  KONFIGURATION
   Training Timesteps pro Quartal: 30000
   Geschätzte Dauer pro Quartal: 9 Minuten
   Modelle: PPO, DDPG, TD3
📊 WALK-FORWARD ROLLING-WINDOW-EVALUIERUNG
   Historischer Start: 2008-01-01
   Test-Zeitraum: 2022-01-01 bis 2025-11-01
   Anzahl Test-Quartale: 16
   Modelle: ['ppo', 'ddpg', 'td3']
   Training Timesteps pro Quartal: 30000

   ⚠️  WICHTIG: Modelle werden in JEDEM Quartal NEU TRAINIERT!

🔍 QUARTAL 1/16: Q1: 2022-01-01 bis 2022-04-01
   📚 Training-Fenster: 2008-01-01 bis 2021-12-31
   🎯 Test-Fenster: 2022-01-01 bis 2022-04-01
   ✅ Training: 3273 Handelstage | Test: 63 Handelstage

   🤖 Modell: PPO
      [1/2] Trainiere PPO auf 3273 Handelstagen...
{'n_steps': 2048, 'ent_coef': 0.01, 'learning_rate': 0.00025, 'batch_size': 64}
Using cpu device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
----------------------------------
| time/              |           |
|    fps             | 1244      |
|    iterations      | 1         |
|  

Traceback (most recent call last):
  File "/var/folders/m8/v5f6mgds38vc772w_62zc7qm0000gn/T/ipykernel_2313/2868726879.py", line 246, in run_walk_forward_evaluation
    results = train_and_evaluate_walk_forward(
        model_type=model_name,
    ...<5 lines>...
        timesteps=training_timesteps
    )
  File "/var/folders/m8/v5f6mgds38vc772w_62zc7qm0000gn/T/ipykernel_2313/2868726879.py", line 104, in train_and_evaluate_walk_forward
    action, _states = trained_model.predict(obs, deterministic=True)
                      ~~~~~~~~~~~~~~~~~~~~~^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/simonpapekyan/DRL/.venv/lib/python3.13/site-packages/stable_baselines3/common/base_class.py", line 557, in predict
    return self.policy.predict(observation, state, episode_start, deterministic)
           ~~~~~~~~~~~~~~~~~~~^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/simonpapekyan/DRL/.venv/lib/python3.13/site-packages/stable_baselines3/common/policies.py", line 357, in predict
    

Logging Error: 'rollout_buffer'
Logging Error: 'rollout_buffer'
Logging Error: 'rollout_buffer'
Logging Error: 'rollout_buffer'
Logging Error: 'rollout_buffer'
Logging Error: 'rollout_buffer'
Logging Error: 'rollout_buffer'
Logging Error: 'rollout_buffer'
Logging Error: 'rollout_buffer'
Logging Error: 'rollout_buffer'
Logging Error: 'rollout_buffer'
Logging Error: 'rollout_buffer'
Logging Error: 'rollout_buffer'
Logging Error: 'rollout_buffer'
Logging Error: 'rollout_buffer'
Logging Error: 'rollout_buffer'
Logging Error: 'rollout_buffer'
Logging Error: 'rollout_buffer'
Logging Error: 'rollout_buffer'
Logging Error: 'rollout_buffer'
Logging Error: 'rollout_buffer'
Logging Error: 'rollout_buffer'
Logging Error: 'rollout_buffer'
Logging Error: 'rollout_buffer'
Logging Error: 'rollout_buffer'
Logging Error: 'rollout_buffer'
Logging Error: 'rollout_buffer'
Logging Error: 'rollout_buffer'
Logging Error: 'rollout_buffer'
Logging Error: 'rollout_buffer'
Logging Error: 'rollout_buffer'
Logging 

Traceback (most recent call last):
  File "/var/folders/m8/v5f6mgds38vc772w_62zc7qm0000gn/T/ipykernel_2313/2868726879.py", line 246, in run_walk_forward_evaluation
    results = train_and_evaluate_walk_forward(
        model_type=model_name,
    ...<5 lines>...
        timesteps=training_timesteps
    )
  File "/var/folders/m8/v5f6mgds38vc772w_62zc7qm0000gn/T/ipykernel_2313/2868726879.py", line 104, in train_and_evaluate_walk_forward
    action, _states = trained_model.predict(obs, deterministic=True)
                      ~~~~~~~~~~~~~~~~~~~~~^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/simonpapekyan/DRL/.venv/lib/python3.13/site-packages/stable_baselines3/common/base_class.py", line 557, in predict
    return self.policy.predict(observation, state, episode_start, deterministic)
           ~~~~~~~~~~~~~~~~~~~^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/simonpapekyan/DRL/.venv/lib/python3.13/site-packages/stable_baselines3/common/policies.py", line 357, in predict
    

Logging Error: 'rollout_buffer'
Logging Error: 'rollout_buffer'
Logging Error: 'rollout_buffer'
Logging Error: 'rollout_buffer'
Logging Error: 'rollout_buffer'
Logging Error: 'rollout_buffer'
Logging Error: 'rollout_buffer'
Logging Error: 'rollout_buffer'
Logging Error: 'rollout_buffer'
Logging Error: 'rollout_buffer'
Logging Error: 'rollout_buffer'
Logging Error: 'rollout_buffer'
Logging Error: 'rollout_buffer'
Logging Error: 'rollout_buffer'
Logging Error: 'rollout_buffer'
Logging Error: 'rollout_buffer'
Logging Error: 'rollout_buffer'
Logging Error: 'rollout_buffer'
Logging Error: 'rollout_buffer'
Logging Error: 'rollout_buffer'
Logging Error: 'rollout_buffer'
Logging Error: 'rollout_buffer'
Logging Error: 'rollout_buffer'
Logging Error: 'rollout_buffer'
Logging Error: 'rollout_buffer'
Logging Error: 'rollout_buffer'
Logging Error: 'rollout_buffer'
Logging Error: 'rollout_buffer'
Logging Error: 'rollout_buffer'
Logging Error: 'rollout_buffer'
Logging Error: 'rollout_buffer'
Logging 

---

## ✅ Zusammenfassung: Walk-Forward Rolling-Window-Evaluierung

**Was wurde implementiert:**

1. **Echtes Walk-Forward Training**: In jedem Quartal werden die Modelle NEU TRAINIERT auf allen bis dahin verfügbaren Daten (Expanding Window)

2. **Kein Look-Ahead Bias**: Training endet immer BEVOR das Test-Quartal beginnt

3. **Fairer Vergleich**: DRL-Modelle werden jetzt quartalsweise aktualisiert, genau wie Markowitz

4. **data_split Fix**: +1 Tag zum Enddatum hinzugefügt, um sicherzustellen, dass der letzte Tag inkludiert wird

**Zeitaufwand:**
- Bei 30,000 Timesteps: ca. 2-3 Minuten pro Modell pro Quartal
- Für 3 Modelle über ~15 Quartale: ca. 90-135 Minuten Gesamtlaufzeit

**Empfehlung:**
- Für schnelle Tests: 30,000 Timesteps
- Für finale Evaluierung: 50,000 - 100,000 Timesteps

---


### Visualisierung der Rolling-Window-Ergebnisse

Jetzt visualisieren wir die Performance der verschiedenen Modelle über die einzelnen Quartale hinweg.


In [ ]:
# Visualisierung der Walk-Forward Rolling-Window-Ergebnisse
# Alle Plots mit deutschen Beschriftungen

import matplotlib.pyplot as plt
import seaborn as sns

# Überprüfe ob Ergebnisse vorhanden sind
if df_rolling_results.empty:
    print("⚠️  Keine Ergebnisse zum Visualisieren vorhanden!")
else:
    # Setze das Design für schönere Plots
    sns.set_style("whitegrid")
    plt.rcParams['figure.figsize'] = (16, 10)
    
    # Erstelle ein großes Figure mit mehreren Subplots
    fig, axes = plt.subplots(2, 2, figsize=(18, 12))
    fig.suptitle('Walk-Forward Rolling-Window: DRL-Agenten Performance pro Quartal\n' +
                 '(Modelle werden quartalsweise NEU TRAINIERT)', 
                 fontsize=16, fontweight='bold', y=0.995)

    # Plot 1: Rendite pro Quartal
    ax1 = axes[0, 0]
    for model in df_rolling_results['Modell'].unique():
        model_data = df_rolling_results[df_rolling_results['Modell'] == model]
        quarters = range(1, len(model_data) + 1)
        ax1.plot(quarters, model_data['Rendite (%)'].values, marker='o', linewidth=2, 
                 markersize=8, label=model, alpha=0.8)
    
    ax1.set_xlabel('Quartal', fontsize=12, fontweight='bold')
    ax1.set_ylabel('Rendite (%)', fontsize=12, fontweight='bold')
    ax1.set_title('Quartalsweise Rendite', fontsize=14, fontweight='bold', pad=10)
    ax1.legend(loc='best', fontsize=10)
    ax1.grid(True, alpha=0.3)
    ax1.axhline(y=0, color='red', linestyle='--', linewidth=1, alpha=0.5)

    # Plot 2: Sharpe Ratio pro Quartal
    ax2 = axes[0, 1]
    for model in df_rolling_results['Modell'].unique():
        model_data = df_rolling_results[df_rolling_results['Modell'] == model]
        quarters = range(1, len(model_data) + 1)
        ax2.plot(quarters, model_data['Sharpe Ratio'].values, marker='s', linewidth=2, 
                 markersize=8, label=model, alpha=0.8)
    
    ax2.set_xlabel('Quartal', fontsize=12, fontweight='bold')
    ax2.set_ylabel('Sharpe Ratio', fontsize=12, fontweight='bold')
    ax2.set_title('Quartalsweise Sharpe Ratio', fontsize=14, fontweight='bold', pad=10)
    ax2.legend(loc='best', fontsize=10)
    ax2.grid(True, alpha=0.3)
    ax2.axhline(y=0, color='red', linestyle='--', linewidth=1, alpha=0.5)
    
    # Plot 3: Maximum Drawdown pro Quartal
    ax3 = axes[1, 0]
    for model in df_rolling_results['Modell'].unique():
        model_data = df_rolling_results[df_rolling_results['Modell'] == model]
        quarters = range(1, len(model_data) + 1)
        ax3.plot(quarters, model_data['Max Drawdown (%)'].values, marker='^', linewidth=2, 
                 markersize=8, label=model, alpha=0.8)
    
    ax3.set_xlabel('Quartal', fontsize=12, fontweight='bold')
    ax3.set_ylabel('Max Drawdown (%)', fontsize=12, fontweight='bold')
    ax3.set_title('Quartalsweise Max. Verlust (Drawdown)', fontsize=14, fontweight='bold', pad=10)
    ax3.legend(loc='best', fontsize=10)
    ax3.grid(True, alpha=0.3)
    
    # Plot 4: Volatilität pro Quartal
    ax4 = axes[1, 1]
    for model in df_rolling_results['Modell'].unique():
        model_data = df_rolling_results[df_rolling_results['Modell'] == model]
        quarters = range(1, len(model_data) + 1)
        ax4.plot(quarters, model_data['Volatilität (%)'].values, marker='d', linewidth=2, 
                 markersize=8, label=model, alpha=0.8)
    
    ax4.set_xlabel('Quartal', fontsize=12, fontweight='bold')
    ax4.set_ylabel('Volatilität (%)', fontsize=12, fontweight='bold')
    ax4.set_title('Quartalsweise Volatilität (annualisiert)', fontsize=14, fontweight='bold', pad=10)
    ax4.legend(loc='best', fontsize=10)
    ax4.grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.savefig('results/walk_forward_performance.png', dpi=300, bbox_inches='tight')
    print("💾 Visualisierung gespeichert unter: results/walk_forward_performance.png")
    plt.show()
    
    # Zusätzliche Statistik: Durchschnittswerte über alle Quartale
    print("\n" + "="*80)
    print("📊 DURCHSCHNITTLICHE PERFORMANCE ÜBER ALLE QUARTALE")
    print("="*80)
    
    summary_stats = df_rolling_results.groupby('Modell').agg({
        'Rendite (%)': ['mean', 'std', 'min', 'max'],
        'Sharpe Ratio': ['mean', 'std'],
        'Max Drawdown (%)': ['mean', 'min'],
        'Volatilität (%)': ['mean']
    }).round(3)
    
    print(summary_stats)
    
    # Speichere auch die Zusammenfassung
    summary_stats.to_csv('results/walk_forward_summary.csv')
    print("\n💾 Zusammenfassung gespeichert unter: results/walk_forward_summary.csv")


### Vergleich mit Baselines


In [ ]:
# Vergleich der DRL-Agenten mit Baseline-Strategien über Rolling Windows
# Berechne Baselines für jedes Quartal

def evaluate_baseline_on_window(baseline_type, prices_window, initial_capital):
    """
    Evaluiert eine Baseline-Strategie auf einem Datenfenster.
    
    Args:
        baseline_type: "equal_weight" oder "markowitz"
        prices_window: Preisdaten für dieses Fenster
        initial_capital: Startkapital
    
    Returns:
        Dictionary mit Performance-Metriken
    """
    returns_window = prices_window.pct_change().dropna(how='all')
    
    if baseline_type == "equal_weight":
        # Equal-Weight: Alle Aktien gleich gewichten
        n_stocks = len(prices_window.columns)
        weights = pd.Series(1.0 / n_stocks, index=prices_window.columns)
        
    elif baseline_type == "markowitz":
        # Markowitz: Mean-Variance Optimierung mit Lookback
        try:
            mu = expected_returns.mean_historical_return(prices_window, frequency=252)
            S = risk_models.sample_cov(prices_window, frequency=252)
            ef = EfficientFrontier(mu, S)
            ef.max_sharpe()
            weights_dict = ef.clean_weights()
            weights = pd.Series(weights_dict)
        except:
            # Fallback auf Equal-Weight bei Optimierungsfehler
            n_stocks = len(prices_window.columns)
            weights = pd.Series(1.0 / n_stocks, index=prices_window.columns)
    
    # Berechne Portfolio-Returns
    portfolio_returns = (returns_window * weights).sum(axis=1)
    
    # Berechne Portfolio-Werte
    portfolio_values = initial_capital * (1 + portfolio_returns).cumprod()
    
    # Performance-Metriken
    initial_value = initial_capital
    final_value = portfolio_values.iloc[-1]
    total_return = (final_value - initial_value) / initial_value
    
    if len(portfolio_returns) > 1 and portfolio_returns.std() != 0:
        sharpe_ratio = (portfolio_returns.mean() / portfolio_returns.std()) * np.sqrt(252)
    else:
        sharpe_ratio = 0.0
    
    cumulative_returns = portfolio_values / initial_value
    running_max = np.maximum.accumulate(cumulative_returns)
    drawdown = (cumulative_returns - running_max) / running_max
    max_drawdown = np.min(drawdown)
    
    volatility = portfolio_returns.std() * np.sqrt(252) if len(portfolio_returns) > 1 else 0.0
    
    return {
        'total_return': total_return,
        'sharpe_ratio': sharpe_ratio,
        'max_drawdown': max_drawdown,
        'volatility': volatility,
        'num_days': len(portfolio_values)
    }


# Führe Baseline-Evaluierung durch (nur wenn DRL-Ergebnisse vorhanden)
if not df_rolling_results.empty:
    print("\n" + "="*80)
    print("🔄 EVALUIERE BASELINE-STRATEGIEN MIT ROLLING WINDOW")
    print("="*80)
    
    test_windows = create_quarterly_windows(trade_start_date, trade_end_date)
    baseline_results = []
    
    for i, (window_start, window_end) in enumerate(test_windows):
        window_name = f"Q{i+1}: {window_start.strftime('%Y-%m-%d')} bis {window_end.strftime('%Y-%m-%d')}"
        print(f"\n🔍 {window_name}")
        
        # FIX: Füge +1 Tag zum Enddatum hinzu (wie bei DRL-Modellen)
        window_end_safe = window_end + timedelta(days=1)
        
        # Filtere Preisdaten für dieses Fenster
        prices_window = prices_trade.loc[
            (prices_trade.index >= window_start) & 
            (prices_trade.index < window_end_safe)
        ]
        
        if len(prices_window) < 10:
            print(f"   ⚠️ Überspringe (zu wenig Daten)")
            continue
    
        # Evaluiere Equal-Weight
        try:
            eq_results = evaluate_baseline_on_window("equal_weight", prices_window, initial_capital)
            baseline_results.append({
                'Quartal': window_name,
                'Modell': 'EQUAL-WEIGHT',
                'Rendite (%)': eq_results['total_return'] * 100,
                'Sharpe Ratio': eq_results['sharpe_ratio'],
                'Max Drawdown (%)': eq_results['max_drawdown'] * 100,
                'Volatilität (%)': eq_results['volatility'] * 100,
                'Handelstage': eq_results['num_days']
            })
            print(f"   ✅ Equal-Weight: Rendite {eq_results['total_return']:.2%}")
        except Exception as e:
            print(f"   ❌ Equal-Weight Fehler: {e}")
        
        # Evaluiere Markowitz
        try:
            mw_results = evaluate_baseline_on_window("markowitz", prices_window, initial_capital)
            baseline_results.append({
                'Quartal': window_name,
                'Modell': 'MARKOWITZ',
                'Rendite (%)': mw_results['total_return'] * 100,
                'Sharpe Ratio': mw_results['sharpe_ratio'],
                'Max Drawdown (%)': mw_results['max_drawdown'] * 100,
                'Volatilität (%)': mw_results['volatility'] * 100,
                'Handelstage': mw_results['num_days']
            })
            print(f"   ✅ Markowitz: Rendite {mw_results['total_return']:.2%}")
        except Exception as e:
            print(f"   ❌ Markowitz Fehler: {e}")
    
    # Kombiniere DRL und Baseline Ergebnisse
    df_baseline_results = pd.DataFrame(baseline_results)
    df_all_results = pd.concat([df_rolling_results, df_baseline_results], ignore_index=True)
    
    print("\n" + "="*80)
    print("📊 VERGLEICH: WALK-FORWARD DRL-AGENTEN vs. BASELINES")
    print("="*80)
    
    # Gruppiere nach Modell und berechne Durchschnittswerte
    comparison = df_all_results.groupby('Modell').agg({
        'Rendite (%)': 'mean',
        'Sharpe Ratio': 'mean',
        'Max Drawdown (%)': 'mean',
        'Volatilität (%)': 'mean'
    }).round(3).sort_values('Sharpe Ratio', ascending=False)
    
    print("\n📈 Durchschnittliche Performance über alle Quartale:")
    print(comparison)
    
    # Speichere die vollständigen Ergebnisse
    df_all_results.to_csv('results/walk_forward_comparison_all.csv', index=False)
    print("\n💾 Alle Ergebnisse gespeichert unter: results/walk_forward_comparison_all.csv")
else:
    print("\n⚠️  Keine DRL-Ergebnisse vorhanden - überspringe Baseline-Vergleich.")


In [ ]:
# Finale Vergleichsvisualisierung: Walk-Forward DRL-Agenten vs. Baselines
# Erstelle umfassende Vergleichsplots

# Nur ausführen wenn alle Ergebnisse vorhanden sind
if not df_rolling_results.empty and 'df_all_results' in locals():
    fig, axes = plt.subplots(2, 2, figsize=(20, 14))
    fig.suptitle('Walk-Forward Rolling-Window: DRL-Agenten vs. Baseline-Strategien\n' +
                 '(DRL-Modelle werden quartalsweise NEU TRAINIERT)', 
                 fontsize=18, fontweight='bold', y=0.995)
    
    # Farbschema für bessere Unterscheidung
    colors = {
        'PPO': '#1f77b4',
        'DDPG': '#ff7f0e', 
        'TD3': '#2ca02c',
        'EQUAL-WEIGHT': '#d62728',
        'MARKOWITZ': '#9467bd'
    }
    
    linestyles = {
        'PPO': '-',
        'DDPG': '-',
        'TD3': '-',
        'EQUAL-WEIGHT': '--',
        'MARKOWITZ': '--'
    }
    
    # Plot 1: Rendite Vergleich
    ax1 = axes[0, 0]
    for model in df_all_results['Modell'].unique():
        model_data = df_all_results[df_all_results['Modell'] == model]
        quarters = range(1, len(model_data) + 1)
        ax1.plot(quarters, model_data['Rendite (%)'].values, 
                 marker='o', linewidth=2.5, markersize=7,
                 label=model, alpha=0.8,
                 color=colors.get(model, 'gray'),
                 linestyle=linestyles.get(model, '-'))
    
    ax1.set_xlabel('Quartal', fontsize=13, fontweight='bold')
    ax1.set_ylabel('Rendite (%)', fontsize=13, fontweight='bold')
    ax1.set_title('Quartalsweise Rendite - Alle Strategien', fontsize=15, fontweight='bold', pad=10)
    ax1.legend(loc='best', fontsize=11, framealpha=0.9)
    ax1.grid(True, alpha=0.3)
    ax1.axhline(y=0, color='black', linestyle=':', linewidth=1.5, alpha=0.5)
    
    # Plot 2: Sharpe Ratio Vergleich
    ax2 = axes[0, 1]
    for model in df_all_results['Modell'].unique():
        model_data = df_all_results[df_all_results['Modell'] == model]
        quarters = range(1, len(model_data) + 1)
        ax2.plot(quarters, model_data['Sharpe Ratio'].values,
                 marker='s', linewidth=2.5, markersize=7,
                 label=model, alpha=0.8,
                 color=colors.get(model, 'gray'),
                 linestyle=linestyles.get(model, '-'))
    
    ax2.set_xlabel('Quartal', fontsize=13, fontweight='bold')
    ax2.set_ylabel('Sharpe Ratio', fontsize=13, fontweight='bold')
    ax2.set_title('Quartalsweise Sharpe Ratio - Alle Strategien', fontsize=15, fontweight='bold', pad=10)
    ax2.legend(loc='best', fontsize=11, framealpha=0.9)
    ax2.grid(True, alpha=0.3)
    ax2.axhline(y=0, color='black', linestyle=':', linewidth=1.5, alpha=0.5)
    
    # Plot 3: Box Plot - Rendite Verteilung
    ax3 = axes[1, 0]
    df_all_results.boxplot(column='Rendite (%)', by='Modell', ax=ax3, 
                            patch_artist=True, grid=False)
    ax3.set_xlabel('Strategie', fontsize=13, fontweight='bold')
    ax3.set_ylabel('Rendite (%)', fontsize=13, fontweight='bold')
    ax3.set_title('Rendite-Verteilung über alle Quartale', fontsize=15, fontweight='bold', pad=10)
    ax3.get_figure().suptitle('')  # Entferne automatischen Titel
    plt.setp(ax3.xaxis.get_majorticklabels(), rotation=45, ha='right')
    ax3.axhline(y=0, color='red', linestyle='--', linewidth=1, alpha=0.5)
    
    # Plot 4: Bar Chart - Durchschnittliche Performance
    ax4 = axes[1, 1]
    avg_performance = df_all_results.groupby('Modell')['Sharpe Ratio'].mean().sort_values(ascending=False)
    bars = ax4.barh(range(len(avg_performance)), avg_performance.values, 
                    color=[colors.get(m, 'gray') for m in avg_performance.index],
                    alpha=0.8, edgecolor='black', linewidth=1.5)
    ax4.set_yticks(range(len(avg_performance)))
    ax4.set_yticklabels(avg_performance.index, fontsize=11, fontweight='bold')
    ax4.set_xlabel('Durchschnittliche Sharpe Ratio', fontsize=13, fontweight='bold')
    ax4.set_title('Ranking nach durchschnittlicher Sharpe Ratio', fontsize=15, fontweight='bold', pad=10)
    ax4.grid(True, alpha=0.3, axis='x')
    ax4.axvline(x=0, color='black', linestyle=':', linewidth=1.5, alpha=0.5)
    
    # Füge Werte zu den Bars hinzu
    for i, (bar, val) in enumerate(zip(bars, avg_performance.values)):
        ax4.text(val + 0.02, i, f'{val:.3f}', va='center', fontsize=10, fontweight='bold')
    
    plt.tight_layout()
    plt.savefig('results/walk_forward_comparison_complete.png', dpi=300, bbox_inches='tight')
    print("💾 Finale Vergleichsvisualisierung gespeichert unter: results/walk_forward_comparison_complete.png")
    plt.show()
    
    # Erstelle eine detaillierte Statistik-Tabelle
    print("\n" + "="*80)
    print("📊 DETAILLIERTE STATISTIK-ÜBERSICHT")
    print("="*80)
    
    detailed_stats = df_all_results.groupby('Modell').agg({
        'Rendite (%)': ['mean', 'std', 'min', 'max', 'median'],
        'Sharpe Ratio': ['mean', 'std', 'min', 'max'],
        'Max Drawdown (%)': ['mean', 'min'],
        'Volatilität (%)': ['mean', 'std']
    }).round(3)
    
    detailed_stats.columns = ['_'.join(col).strip() for col in detailed_stats.columns.values]
    print(detailed_stats)
    
    # Finde das beste Modell
    best_sharpe_model = comparison.index[0]
    best_return_model = df_all_results.groupby('Modell')['Rendite (%)'].mean().idxmax()
    
    print(f"\n🏆 BESTE STRATEGIEN:")
    print(f"   📈 Höchste durchschnittliche Sharpe Ratio: {best_sharpe_model}")
    print(f"   💰 Höchste durchschnittliche Rendite: {best_return_model}")
    
    # Konsistenz-Analyse: Wie oft war jedes Modell positiv?
    print(f"\n📈 KONSISTENZ-ANALYSE (Positive Quartale):")
    for model in df_all_results['Modell'].unique():
        model_data = df_all_results[df_all_results['Modell'] == model]
        positive_quarters = (model_data['Rendite (%)'] > 0).sum()
        total_quarters = len(model_data)
        consistency_rate = (positive_quarters / total_quarters) * 100
        print(f"   {model:15s}: {positive_quarters}/{total_quarters} positiv ({consistency_rate:.1f}%)")
else:
    print("\n⚠️  Keine Ergebnisse zum Visualisieren - führen Sie zuerst die Walk-Forward Evaluierung aus!")
